<a href="https://colab.research.google.com/github/PeopleOfPlay/transforCAOCH/blob/main/generate_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
title = "Le Dernier samouraï"
meta = "Historique, Guerre"
director = "Edward Zwick"
cast = "Tom Cruise, Billy Connolly, Tony Goldwyn"

In [21]:
!pip install transformers

In [22]:
import torch
import gdown
from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel

In [23]:
url = "https://drive.google.com/drive/folders/1rlRM4geoYCGHnbGv5IOADg7FB4rtK1Vq?usp=sharing"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/model_save_v2/added_tokens.json',
 '/content/model_save_v2/config.json',
 '/content/model_save_v2/merges.txt',
 '/content/model_save_v2/pytorch_model.bin',
 '/content/model_save_v2/special_tokens_map.json',
 '/content/model_save_v2/tokenizer_config.json',
 '/content/model_save_v2/vocab.json']

In [24]:
path = "model_save_v2"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [25]:
tokenizer = GPT2Tokenizer.from_pretrained(path, bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>', local_files_only=True)
configuration = GPT2Config.from_pretrained(path, output_hidden_states=False, local_files_only=True)
model = GPT2LMHeadModel.from_pretrained(path, config=configuration, local_files_only=True)
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [26]:
model.to(device)
model.eval()
prompt = f"<|startoftext|>{title} {meta} {director} {cast} ::"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

with torch.no_grad():
    sample_outputs = model.generate(
        generated.to(device), 
        do_sample=True,   
        top_k=50, 
        max_length=500,
        top_p=0.95, 
        num_return_sequences=5
    )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [27]:
for i, sample_output in enumerate(sample_outputs):
    comment = tokenizer.decode(sample_output, skip_special_tokens=True).split('::')[1]
    print(f"{i}: {comment}\n\n")

0:  Le Dernier samouraï : Un film de guerre des plus percutants et originaux qui déjà bien la maitrise du film : visuellement soignée, travaillé et surtout bien mené. Ce film nous fait voyages dans l’action et ça c’est plutôt une véritable expérience grâce a ces répliques qui rend le film moins bien mise en avant car j’adore les gros répliques de guerre. Et la réalisation est honorable car on a le droit a de très beaux décors, costumes et paysages sublimes. Et le film est assez malsain (la démonstration) car ce film nous réserve quand même une bonne ambiance d’émotion et de sensation : c’est très bien. Ensuite, le scénario est assez spécial et nous plonge dans un camp de guerre des plus fort de l’amitié : dans cette guerre, il va rentrer dans les camps dans un camp de guerre par une femme qui veut s’occupé de ses autres… Et donc, le mari des deux personnages est une sorte de société mais elle va devoir tué pour qu’elle soit plus humain qui va crescendo dans l’avant. Donc voila, scénari